In [1]:
from copy import deepcopy
import torch
from torch import optim
from torch import nn
from torch.nn import functional as F
from scipy import sparse
import pandas as pd
import os
import sys
import bottleneck as bn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

<b><h3>Model

In [2]:
def swish(x):
    return x.mul(torch.sigmoid(x))

def log_norm_pdf(x, mu, logvar):
    return -0.5*(logvar + np.log(2 * np.pi) + (x - mu).pow(2) / logvar.exp())

class CompositePrior(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, mixture_weights=[3/20, 3/4, 1/10]):
        super(CompositePrior, self).__init__()
        
        self.mixture_weights = mixture_weights
        
        self.mu_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.mu_prior.data.fill_(0)
        
        self.logvar_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_prior.data.fill_(0)
        
        self.logvar_uniform_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_uniform_prior.data.fill_(10)
        
        self.encoder_old = Encoder(hidden_dim, latent_dim, input_dim)
        self.encoder_old.requires_grad_(False)
        
    def forward(self, x, z):
        post_mu, post_logvar = self.encoder_old(x, 0)
        
        stnd_prior = log_norm_pdf(z, self.mu_prior, self.logvar_prior)
        post_prior = log_norm_pdf(z, post_mu, post_logvar)
        unif_prior = log_norm_pdf(z, self.mu_prior, self.logvar_uniform_prior)
        
        gaussians = [stnd_prior, post_prior, unif_prior]
        gaussians = [g.add(np.log(w)) for g, w in zip(gaussians, self.mixture_weights)]
        
        density_per_gaussian = torch.stack(gaussians, dim=-1)
                
        return torch.logsumexp(density_per_gaussian, dim=-1)

class Encoder(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, eps=1e-1):
        super(Encoder, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.ln4 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.ln5 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x, dropout_rate):
        norm = x.pow(2).sum(dim=-1).sqrt()
        x = x / norm[:, None]
    
        x = F.dropout(x, p=dropout_rate, training=self.training)
        
        h1 = self.ln1(swish(self.fc1(x)))
        h2 = self.ln2(swish(self.fc2(h1) + h1))
        h3 = self.ln3(swish(self.fc3(h2) + h1 + h2))
        h4 = self.ln4(swish(self.fc4(h3) + h1 + h2 + h3))
        h5 = self.ln5(swish(self.fc5(h4) + h1 + h2 + h3 + h4))
        return self.fc_mu(h5), self.fc_logvar(h5)
    

class VAE(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim):
        super(VAE, self).__init__()

        self.encoder = Encoder(hidden_dim, latent_dim, input_dim)
        self.prior = CompositePrior(hidden_dim, latent_dim, input_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)
        
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, user_ratings, beta=None, gamma=1, dropout_rate=0.5, calculate_loss=True):
        mu, logvar = self.encoder(user_ratings, dropout_rate=dropout_rate)    
        z = self.reparameterize(mu, logvar)
        x_pred = self.decoder(z)
        
        if calculate_loss:
            if gamma:
                norm = user_ratings.sum(dim=-1)
                kl_weight = gamma * norm
            elif beta:
                kl_weight = beta

            mll = (F.log_softmax(x_pred, dim=-1) * user_ratings).sum(dim=-1).mean()
            kld = (log_norm_pdf(z, mu, logvar) - self.prior(user_ratings, z)).sum(dim=-1).mul(kl_weight).mean()
            negative_elbo = -(mll - kld)
            
            return (mll, kld), negative_elbo
            
        else:
            return x_pred

    def update_prior(self):
        self.prior.encoder_old.load_state_dict(deepcopy(self.encoder.state_dict()))

<b><h3>Utils

In [3]:
# based on https://github.com/dawenl/vae_cf
def load_train_data(csv_file, n_items, n_users, global_indexing=False):
    tp = pd.read_csv(csv_file)
    
    n_users = n_users if global_indexing else tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

def load_tr_te_data(csv_file_tr, csv_file_te, n_items, n_users, global_indexing=False):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    if global_indexing:
        start_idx = 0
        end_idx = len(unique_uid) - 1
    else:
        start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
        end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

def get_data(dataset, global_indexing=False):
    unique_sid = list()
    with open(os.path.join(dataset, 'unique_sid.txt'), 'r') as f:
        for line in f:
            unique_sid.append(line.strip())
    
    unique_uid = list()
    with open(os.path.join(dataset, 'unique_uid.txt'), 'r') as f:
        for line in f:
            unique_uid.append(line.strip())
            
    n_items = len(unique_sid)
    n_users = len(unique_uid)
    
    train_data = load_train_data(os.path.join(dataset, 'train.csv'), n_items, n_users, global_indexing=global_indexing)


    vad_data_tr, vad_data_te = load_tr_te_data(os.path.join(dataset, 'validation_tr.csv'),
                                               os.path.join(dataset, 'validation_te.csv'),
                                               n_items, n_users, 
                                               global_indexing=global_indexing)

    test_data_tr, test_data_te = load_tr_te_data(os.path.join(dataset, 'test_tr.csv'),
                                                 os.path.join(dataset, 'test_te.csv'),
                                                 n_items, n_users, 
                                                 global_indexing=global_indexing)
    
    data = train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te
    data = (x.astype('float32') for x in data)
    
    return data

def ndcg(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis], idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum() for n in heldout_batch.getnnz(axis=1)])

    # Avoid division by zero: Set NDCG to 0 where IDCG is 0
    valid_idx = IDCG != 0
    NDCG = np.zeros(batch_users)
    NDCG[valid_idx] = DCG[valid_idx] / IDCG[valid_idx]

    return NDCG


def recall(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    intersect = np.logical_and(X_true_binary, X_pred_binary).sum(axis=1).astype(np.float32)
    
    relevant = X_true_binary.sum(axis=1)
    
    # Avoid division by zero: if the denominator is zero, recall can be set to zero (or some other value)
    recall = np.where(relevant > 0, intersect / relevant, 0.0)
    return recall


def precision(X_pred, heldout_batch, k=100):
    '''
    Precision@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    - X_pred: Predictive score matrix, where each row represents a user and each column represents an item
    - heldout_batch: Real user-item interaction data, again, each row represents a user and each column represents an item
    - k: Number of top recommendations considered
    '''
    batch_users = X_pred.shape[0]
    
    # Find the top k highest predicted scores for each user using argpartition
    idx = bn.argpartition(-X_pred, k, axis=1)
    # Create a matrix that is all False and then set the top k highest scores to True
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    # Converts heldout_batch to a binary format indicating whether the user is interested in the item or not
    X_true_binary = (heldout_batch > 0).toarray()
    # Calculate the number of correct predictions for each user
    true_positives = np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)
    # Calculate and return precision@k
    precision_at_k = true_positives.astype(np.float32) / k
    return precision_at_k

<b><h3>Dataset

In [4]:
processed_dataset = r'C:\Users\FOMO\Desktop\Proj\Dataset\Douban\processed_data_2'

<b><h3>Set up

In [5]:
hidden_dim = 600
latent_dim = 200
batch_size = 500
beta = 0.5
gamma = 0.035
n_epochs = 50  # 训练周期数
not_alternating = False  # 是否使用交替训练
n_enc_epochs = 3  # 编码器训练周期数
n_dec_epochs = 1  # 解码器训练周期数

In [6]:
data = get_data(processed_dataset)
train_data, valid_in_data, valid_out_data, test_in_data, test_out_data = data

In [7]:
seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [8]:
device = torch.device("cuda:0")

In [9]:
def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
    assert 0 < samples_perc_per_epoch <= 1
    
    total_samples = data_in.shape[0]
    samples_per_epoch = int(total_samples * samples_perc_per_epoch)
    
    if shuffle:
        idxlist = np.arange(total_samples)
        np.random.shuffle(idxlist)
        idxlist = idxlist[:samples_per_epoch]
    else:
        idxlist = np.arange(samples_per_epoch)
    
    for st_idx in range(0, samples_per_epoch, batch_size):
        end_idx = min(st_idx + batch_size, samples_per_epoch)
        idx = idxlist[st_idx:end_idx]

        yield Batch(device, idx, data_in, data_out)

class Batch:
    def __init__(self, device, idx, data_in, data_out=None):
        self._device = device
        self._idx = idx
        self._data_in = data_in
        self._data_out = data_out
    
    def get_idx(self):
        return self._idx
    
    def get_idx_to_dev(self):
        return torch.LongTensor(self.get_idx()).to(self._device)
        
    def get_ratings(self, is_out=False):
        data = self._data_out if is_out else self._data_in
        return data[self._idx]
    
    def get_ratings_to_dev(self, is_out=False):
        return torch.Tensor(
            self.get_ratings(is_out).toarray()
        ).to(self._device)

def evaluate(model, data_in, data_out, metrics, samples_perc_per_epoch=1, batch_size=500):
    metrics = deepcopy(metrics)
    model.eval()
    
    for m in metrics:
        m['score'] = []
    
    for batch in generate(batch_size=batch_size,
                          device=device,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):
        
        ratings_in = batch.get_ratings_to_dev()
        ratings_out = batch.get_ratings(is_out=True)
    
        ratings_pred = model(ratings_in, calculate_loss=False).cpu().detach().numpy()
        
        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf
            
        for m in metrics:
            m['score'].append(m['metric'](ratings_pred, ratings_out, k=m['k']))

    for m in metrics:
        m['score'] = np.concatenate(m['score']).mean()
        
    return [x['score'] for x in metrics]

def run(model, opts, train_data, batch_size, n_epochs, beta, gamma, dropout_rate):
    model.train()
    for epoch in range(n_epochs):
        for batch in generate(batch_size=batch_size, device=device, data_in=train_data, shuffle=True):
            ratings = batch.get_ratings_to_dev()

            for optimizer in opts:
                optimizer.zero_grad()
                
            _, loss = model(ratings, beta=beta, gamma=gamma, dropout_rate=dropout_rate)
            loss.backward()
            
            for optimizer in opts:
                optimizer.step()

In [10]:
model_kwargs = {
    'hidden_dim': hidden_dim,
    'latent_dim': latent_dim,
    'input_dim': train_data.shape[1]
}
metrics = [{'metric': ndcg, 'k': 100}]

In [11]:
best_ndcg = -np.inf
train_scores, valid_scores = [], []

In [12]:
model = VAE(**model_kwargs).to(device)
model_best = VAE(**model_kwargs).to(device)

In [13]:
learning_kwargs = {
    'model': model,
    'train_data': train_data,
    'batch_size': batch_size,
    'beta': beta,
    'gamma': gamma
}

In [14]:
decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

In [15]:
optimizer_encoder = optim.Adam(encoder_params, lr=0.001)
optimizer_decoder = optim.Adam(decoder_params, lr=0.001)

<b><h3>Train the model

In [16]:
for epoch in range(n_epochs):

    if not_alternating:
        run(opts=[optimizer_encoder, optimizer_decoder], n_epochs=1, dropout_rate=0.5, **learning_kwargs)
    else:
        run(opts=[optimizer_encoder], n_epochs=n_enc_epochs, dropout_rate=0.5, **learning_kwargs)
        model.update_prior()
        run(opts=[optimizer_decoder], n_epochs=n_dec_epochs, dropout_rate=0, **learning_kwargs)

    train_scores.append(
        evaluate(model, train_data, train_data, metrics, 0.01)[0]
    )
    valid_scores.append(
        evaluate(model, valid_in_data, valid_out_data, metrics, 1)[0]
    )
    
    if valid_scores[-1] > best_ndcg:
        best_ndcg = valid_scores[-1]
        model_best.load_state_dict(deepcopy(model.state_dict()))
    
    print(f'epoch {epoch} | valid ndcg@100: {valid_scores[-1]:.4f} | ' +
          f'best valid: {best_ndcg:.4f} | train ndcg@100: {train_scores[-1]:.4f}')

epoch 0 | valid ndcg@100: 0.0129 | best valid: 0.0129 | train ndcg@100: 0.0248
epoch 1 | valid ndcg@100: 0.0164 | best valid: 0.0164 | train ndcg@100: 0.0298
epoch 2 | valid ndcg@100: 0.0263 | best valid: 0.0263 | train ndcg@100: 0.0503
epoch 3 | valid ndcg@100: 0.0561 | best valid: 0.0561 | train ndcg@100: 0.1235
epoch 4 | valid ndcg@100: 0.0910 | best valid: 0.0910 | train ndcg@100: 0.2476
epoch 5 | valid ndcg@100: 0.1174 | best valid: 0.1174 | train ndcg@100: 0.3767
epoch 6 | valid ndcg@100: 0.1346 | best valid: 0.1346 | train ndcg@100: 0.4856
epoch 7 | valid ndcg@100: 0.1450 | best valid: 0.1450 | train ndcg@100: 0.5845
epoch 8 | valid ndcg@100: 0.1513 | best valid: 0.1513 | train ndcg@100: 0.6633
epoch 9 | valid ndcg@100: 0.1547 | best valid: 0.1547 | train ndcg@100: 0.7301
epoch 10 | valid ndcg@100: 0.1572 | best valid: 0.1572 | train ndcg@100: 0.7809
epoch 11 | valid ndcg@100: 0.1562 | best valid: 0.1572 | train ndcg@100: 0.8235
epoch 12 | valid ndcg@100: 0.1576 | best valid: 0.

In [17]:
test_metrics = [
    {'metric': recall, 'k': 5},
    {'metric': recall, 'k': 10},
    {'metric': recall, 'k': 20},
    {'metric': recall, 'k': 50},
    {'metric': recall, 'k': 100},
    {'metric': precision, 'k': 5},
    {'metric': precision, 'k': 10},
    {'metric': precision, 'k': 20},
    {'metric': precision, 'k': 50},
    {'metric': precision, 'k': 100},
    {'metric': ndcg, 'k': 5},
    {'metric': ndcg, 'k': 10},
    {'metric': ndcg, 'k': 20},
    {'metric': ndcg, 'k': 50},
    {'metric': ndcg, 'k': 100}
]

In [18]:
final_scores = evaluate(model_best, test_in_data, test_out_data, test_metrics)

C:\Users\FOMO\AppData\Local\Temp\ipykernel_30772\3763803702.py:104: RuntimeWarning: invalid value encountered in divide
  recall = np.where(relevant > 0, intersect / relevant, 0.0)


In [19]:
for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

recall@5:	0.1214
recall@10:	0.1543
recall@20:	0.1903
recall@50:	0.2463
recall@100:	0.2955
precision@5:	0.0457
precision@10:	0.0308
precision@20:	0.0201
precision@50:	0.0112
precision@100:	0.0070
ndcg@5:	0.1084
ndcg@10:	0.1187
ndcg@20:	0.1294
ndcg@50:	0.1435
ndcg@100:	0.1537
